# User params

In [1]:
bool_debug = False
embedder_name = 'all-MiniLM-L6-v2'  # dim : 384, max_len : 256 (probably too short for some cases)
max_samples = 1_000
bool_debug = False

In [2]:
import os, sys
sys.path.append(os.path.abspath('../src'))
from src.config import BASE_PATH_DATA, PATH_PROJECT
os.chdir(PATH_PROJECT)

file_path_train_raw = os.path.join(BASE_PATH_DATA, 'processed', 'train.parquet')
file_path_test_raw = os.path.join(BASE_PATH_DATA, 'processed', 'test.parquet')

file_path_train_encoded = os.path.join(BASE_PATH_DATA, 'processed', f'df_train_encoded.parquet')
file_path_test_encoded = os.path.join(BASE_PATH_DATA, 'processed', f'df_test_encoded.parquet')


# Data cleaning, enrichment and encoding

In [3]:
import pandas as pd
from src.utils.io.io import load_dataframe

# read json file into dataframe
df_train = load_dataframe(file_path_train_raw)
df_test = load_dataframe(file_path_test_raw)

# limit to wanted sample size, random_state for reproducibility
df_train = df_train.sample(n=max_samples, random_state=42)
df_test = df_test.sample(n=max_samples, random_state=42)

display(df_train.head(1))

2024-08-05 16:15:15,662 - src.config - DEBUG - calling load_dataframe
2024-08-05 16:15:15,662 - src.config - INFO - Current working directory: C:\Users\alexi\WORKSPACE\interview_abwaab\ReviewOutliers
2024-08-05 16:15:16,907 - src.config - DEBUG - calling load_dataframe
2024-08-05 16:15:16,908 - src.config - INFO - Current working directory: C:\Users\alexi\WORKSPACE\interview_abwaab\ReviewOutliers


,rating,title_review,text,images_review,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,...,description,price,images_metadata,videos,store,categories,details,bought_together,subtitle,author
23211,5.0,Easy to get and use,Being able to order the cards online and have ...,[],B007RFFXAS,B007RFFXAS,AENX4E7J65CV3LC73VJWSOXCBKIA,1619197290719,1,True,...,[Amazon.com Gift Cards are the perfect way to ...,None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Amazon,"[Gift Cards, Gift Card Recipients, For Him]","{""Package Dimensions"": ""4.52 x 3.42 x 0.98 inc...",None,None,None


In [4]:
from src.utils.preprocessing.preprocessing import preprocess_data

# encode the data
df_train_encoded = preprocess_data(df_train, training=True)
saved_name = os.path.join(BASE_PATH_DATA, 'processed', 'df_test_enriched.parquet')
df_test_encoded = preprocess_data(df_test, training=False, saved_name=saved_name)

# print some stats
print(f'length of df_train_encoded dataset: {df_train_encoded.shape}')
print(f'length of df_test_encoded dataset: {df_test_encoded.shape}')
display(df_train_encoded.head(1))

# Save the combined dataset (if needed)
df_train_encoded.to_parquet(file_path_train_encoded, index=False)
df_test_encoded.to_parquet(file_path_test_encoded, index=False)

c:\Users\alexi\AppData\Local\pypoetry\Cache\virtualenvs\reviewoutliers-SwZO3ms--py3.12\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-08-05 16:15:22,803 - datasets - INFO - PyTorch version 2.3.1 available.
2024-08-05 16:15:24,019 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2024-08-05 16:15:24,019 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2024-08-05 16:15:24,019 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): huggingface.co:443
2024-08-05 16:15:24,434 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/11" 200 0
2024-08-05 16:15:24,701 - urllib3.connectionpool

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2024-08-05 16:15:58,906 - src.config - DEBUG - calling encode_data
2024-08-05 16:15:58,923 - src.config - DEBUG - calling preprocess_data
2024-08-05 16:15:58,923 - src.config - DEBUG - calling clean_enrich


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2024-08-05 16:16:26,600 - src.config - DEBUG - calling encode_data


length of df_train_encoded dataset: (1000, 139)
length of df_test_encoded dataset: (1000, 139)


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138
0,-0.285459,0.993793,-0.41736,0.794081,0.437534,0.343634,-0.001175,-0.807844,-0.755935,-0.27591,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
